In [1]:
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import tensorflow as tf

In [2]:
data = []
label = []
for path in os.listdir("/Users/arwa/Desktop/Dataset/Fall"):
    for imgpath in os.listdir("/Users/arwa/Desktop/Dataset/Fall/"+path):
        img = cv2.imread("/Users/arwa/Desktop/Dataset/Fall/"+path+"/"+imgpath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (244,244))
        data.append(img)
        label.append(1)
for path in os.listdir("/Users/arwa/Desktop/Dataset/NotFall"):
    for imgpath in os.listdir("/Users/arwa/Desktop/Dataset/NotFall/"+path):
        img = cv2.imread("/Users/arwa/Desktop/Dataset/NotFall/"+path+"/"+imgpath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (244,244))
        data.append(img)
        label.append(0)

In [3]:
data = np.array(data)
label = np.array(label)

In [4]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, label, test_size=0.25,
                                                     stratify=label, random_state=42)

In [5]:
train_gen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

test_gen = ImageDataGenerator()

In [6]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_gen.mean = mean
test_gen.mean = mean

In [7]:
baseModel = ResNet50(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [8]:
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / 25)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [9]:
H = model.fit(
    train_gen.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=test_gen.flow(X_test, Y_test),
    validation_steps=len(X_test) // 32,
    epochs=25)

Epoch 1/25
559/559 [==============================] - 2175s 4s/step - loss: 0.8416 - accuracy: 0.5002 - val_loss: 0.7150 - val_accuracy: 0.4254
Epoch 2/25
559/559 [==============================] - 2137s 4s/step - loss: 0.7453 - accuracy: 0.4930 - val_loss: 0.7064 - val_accuracy: 0.4155
Epoch 3/25
559/559 [==============================] - 2119s 4s/step - loss: 0.7217 - accuracy: 0.4907 - val_loss: 0.7022 - val_accuracy: 0.4316
Epoch 4/25
559/559 [==============================] - 2210s 4s/step - loss: 0.7123 - accuracy: 0.4942 - val_loss: 0.7008 - val_accuracy: 0.4395
Epoch 5/25
559/559 [==============================] - 2335s 4s/step - loss: 0.7070 - accuracy: 0.4945 - val_loss: 0.6996 - val_accuracy: 0.4293
Epoch 6/25
559/559 [==============================] - 2308s 4s/step - loss: 0.7039 - accuracy: 0.4941 - val_loss: 0.6984 - val_accuracy: 0.4484
Epoch 7/25
559/559 [==============================] - 2058s 4s/step - loss: 0.7018 - accuracy: 0.4906 - val_loss: 0.6977 - val_accuracy:

In [12]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [13]:
model.save_weights('model_weights.h5')

In [10]:
model.save('/Users/arwa/Desktop/Test/Model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /Users/arwa/Desktop/Test/Model/assets


In [11]:
from sklearn.externals import joblib
joblib.dump(model, 'modelout_Test')

/Users/arwa/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


TypeError: can't pickle _thread.RLock objects